In [3]:
from glob import glob
import os
import tempfile
import uuid
import numpy as np
from collections import defaultdict
import itertools

from data_processing.hdf5_with_filtering import AnnotatedSpectrumIndex
from data_processing.tokenization_dataloader import TokenizerDataModule

# Generate the vocabulary and save it to an output file.
# The vocabulary is ordered from most frequent to least frequent, and contains all possible
# "single" m/z values (including those not present in the training data), as well as the top n pairs of co-occurring peaks
# 
# Note that this does not contain special characters (pad, mask, etc); these must be initialized
# when loading with the vocabulary class

# This is partially based on the data loading code from Casanovo: https://github.com/Noble-Lab/casanovo/blob/main/casanovo/denovo/model_runner.py


# Discretizes the original floating-point m/z value 
# based on the token size, and rounds to an integer.

# The token size is assumed to start at the integer value,
# so the possible tokens are min_mz + token_ix * token_size

# each original m/z value is rounded to the nearest possible token based on the token size

# e.g. if min_mz is 50 and token_size = 0.2, the possible tokens are 50.0, 50.2, 50.4, ...
# while if min_mz = 50 and token_size = 1, the possible tokens are 50, 51, 52, ...
# To simplify logic dealing with tokens, values are multiplied by 10^decimal places so they are always represented as integers

# For example, for an m/z value of 158.29474:
# token_size = 1 -> 158
# token_size = 0.1 -> 1583
# token_size = 0.2 -> 1582
# token_size = 0.5 -> 1585

# Token size is passed as a string to avoid floating-point arithmetic errors,
# as the output depends on the number of decimal places. It must be passed as a string with at least 
# one value after the decimal place (e.g. to have a token size of 1, str_token_size should be 1.0)
def to_token(mz_float, str_token_size):
    token_size = float(str_token_size)

    discretized_mz = round(mz_float / token_size) * token_size

    num_decimal_places = len(str_token_size.split(".")[1].rstrip("0"))

    return int(discretized_mz * pow(10, num_decimal_places))

# Create the data loader to use for generating the vocabulary
def create_tokenization_dataloader(tmp_dir, data_path, max_charge, batch_size, start_spec_ix, end_spec_ix, max_spectra_per_peptide, peaks_per_spectrum, min_mz, max_mz, min_intensity, remove_precursor_tol, random_seed, train_ratio):
    train_filenames = glob(os.path.join(data_path, '*'))

    data_idx_fn = os.path.join(tmp_dir.name, f"{uuid.uuid4().hex}.hdf5")

    valid_charge = np.arange(1, max_charge + 1)
    data_index = AnnotatedSpectrumIndex(
        data_idx_fn, start_spec_ix=start_spec_ix, end_spec_ix=end_spec_ix, max_spectra_per_peptide=max_spectra_per_peptide, ms_data_files=train_filenames, valid_charge=valid_charge
    )

    dataloader_params = dict(
        batch_size=batch_size,
        n_peaks=peaks_per_spectrum,
        min_mz=min_mz,
        max_mz=max_mz,
        min_intensity=min_intensity,
        remove_precursor_tol=remove_precursor_tol,
        random_seed = random_seed,
        train_ratio = train_ratio
    )

    data_loader = TokenizerDataModule(data_index=data_index, **dataloader_params)

    data_loader.setup()

    return data_loader

# Calculate the frequencies of all single tokens and pair tokens 
# using the provided dataloader. 

# token_size should be a string indicating the granularity of the tokens to return.
# For example, if the m/z values should be rounded to the nearest 0.2 increments, token_size should be "0.2", or for 1.0 increments, "1.0"
def calculate_single_and_pair_token_frequencies(train_data_loader, min_mz, max_mz, token_size):
    # first, calculate the "raw" single token frequencies and frequencies for all pairs
    single_token_frequencies = {}
    for mz_token in np.arange(min_mz, max_mz, float(token_size)):
        single_token_frequencies[to_token(mz_token, token_size)] = 0

    combination_token_frequencies = defaultdict(lambda:0)    

    for batch in train_data_loader:
        spectra = batch[0]
        for spec in spectra:

            # for each discretized m/z value (token) in the spectrum, keep the highest intensity 
            # (if multiple peaks fall into the same "bin", we consider only the one with the higest intensity)
            peaks_for_this_spec = defaultdict(lambda:0)

            for peak in spec:
                if peak[0] == 0:
                    break # we've reached the end padding, so no need to continue processing this spectrum

                mz_token = to_token(peak[0].item(), token_size)
                intensity = peak[1]

                previous_intensity = peaks_for_this_spec[mz_token]

                peaks_for_this_spec[mz_token] = max(intensity, previous_intensity)

            # store the frequency of all single peaks for this spectrum in the overall dict
            for peak in peaks_for_this_spec.keys():
                single_token_frequencies[peak] += 1

            combinations_in_this_spec = itertools.combinations(peaks_for_this_spec.keys(), 2)

            for peak_combination in combinations_in_this_spec:
                combination_token_frequencies[peak_combination] += 1

    return single_token_frequencies, combination_token_frequencies         

    # generate the full vocabularies from the frequencies, and save to the output files.
    # this saves two vocabularies: one containing just the single tokens, and one containing
    # all single tokens as well as the top pair_token_count pair tokens.

    # Each line in the file is a tuple containing the token itself (either a single or pair of m/z values), and its frequency

    # the vocabulary files are sorted by frequency (highest first).
    # For single-only vocabularies, the frequency is simply the number of times that token appeared in the training data.
    # For the vocabulary including pairs, the frequency of single tokens only considers their frequency when they appear on their own (but not as part of the top pair_token_count pair tokens)
def generate_and_save_vocabulary(train_data_loader, single_token_frequencies, pair_token_frequencies, token_size, single_output_file_name, pair_token_count=0, pair_output_file_name=None):

    # first save the singles-only vocabulary
    sorted_single_frequencies = sorted(single_token_frequencies.items(), key=lambda kv: kv[1], reverse=True)
    with open(single_output_file_name, "w") as f:
        for line in sorted_single_frequencies:
            f.write(f"{line}\n")

    # now find the top pair_token_count pairs, update the frequency of their respective singles, and save to the pair vocabulary file
    sorted_pair_frequencies = sorted(pair_token_frequencies.items(), key=lambda kv: kv[1], reverse=True)[:pair_token_count]

    overall_frequencies = defaultdict(lambda:0)

    # make sure we have tokens for all singles values even if they do not appear in the data
    for key in single_token_frequencies.keys():
        overall_frequencies[key] = 0


    # preprocess the list of pair tokens into a map from each single m/z value to the pair tokens that start with it
    mz_to_pairs_starting_with_it = defaultdict(lambda:set())
    for pair_token in sorted_pair_frequencies:
        token = pair_token[0] # the tuple representing the m/z pair, e.g. (244, 279)
        first_mz = token[0]

        mz_to_pairs_starting_with_it[first_mz].add(token)

    for batch in train_data_loader:
        spectra = batch[0]
        for spec in spectra:
            peaks_in_pairs_this_spec = set() # keep track of the peaks that are part of pair tokens so far, to prevent double counting

            peaks_for_spec = set()
            for peak in spec:
                if peak[0] == 0:
                    break # ignore padding
                peaks_for_spec.add(to_token(peak[0].item(), token_size))

            for peak in peaks_for_spec:
                possible_tokens_starting_with_this_mz = mz_to_pairs_starting_with_it[peak]

                for mz_pair in possible_tokens_starting_with_this_mz:
                    second_mz = mz_pair[1]

                    if second_mz in peaks_for_spec:
                        overall_frequencies[mz_pair] += 1
                        peaks_in_pairs_this_spec.add(peak)
                        peaks_in_pairs_this_spec.add(second_mz)

                if peak not in peaks_in_pairs_this_spec:
                    # only increment the count for the single token if not part of any pairs 
                    overall_frequencies[peak] += 1

    # now sort the overall frequencies and save to the file:
    sorted_overall_frequencies = sorted(overall_frequencies.items(), key=lambda kv: kv[1], reverse=True)
    with open(pair_output_file_name, "w") as f:
        for line in sorted_overall_frequencies:
            f.write(f"{line}\n")

ImportError: attempted relative import with no known parent package

In [10]:
tmp_dir = tempfile.TemporaryDirectory()

data_loader = create_tokenization_dataloader(tmp_dir, data_path="/disk/dragon-storage/homes/seber007/data/spectra/massive-kb-lowres/", max_charge=10, batch_size=1000, start_spec_ix=0, end_spec_ix=200_000, max_spectra_per_peptide=1, peaks_per_spectrum=150, min_mz=50.0, max_mz=2500.0, min_intensity=0.01, remove_precursor_tol=2.0, random_seed=42, train_ratio=0.8)

/disk/dragon-storage/homes/seber007/data/spectra/massive-kb-lowres/LIBRARY_AUGMENT-20026e39-download_filtered_…

In [32]:
train_dataloader = data_loader.train_dataloader()
val_dataloader = data_loader.val_dataloader()

In [12]:
single_token_frequencies, pair_token_frequencies = calculate_single_and_pair_token_frequencies(train_data_loader=train_dataloader, min_mz=50.0, max_mz=2500.0, token_size="1.0")

In [62]:
generate_and_save_vocabulary(train_data_loader=train_dataloader, single_token_frequencies=single_token_frequencies, pair_token_frequencies=pair_token_frequencies, token_size="1.0", single_output_file_name="/lclhome/seber007/single-tokens-mar22.txt", pair_token_count=5550, pair_output_file_name="/lclhome/seber007/pair-tokens-mar22.txt")